<a href="https://colab.research.google.com/github/kyt4n/Malaysia-COVID-19-Dataset/blob/main/covid_dataset_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Update Case Data

In [ ]:
!pip install --upgrade -q gspread 

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [ ]:
import json
import tweepy
import pandas as pd
import re

with open("twitter_creds.json", "r") as file:
  creds = json.load(file)

auth = tweepy.OAuthHandler(creds['api_key'], creds['api_sec'])
auth.set_access_token(creds['access_token'], creds['access_sec'])
api = tweepy.API(auth, wait_on_rate_limit=True)

username = '@DGHisham'
count = 300
tweet_id = '1418525491049361411'

tweets = tweepy.Cursor(api.user_timeline,id=username, tweet_mode="extended", since_id = tweet_id,
                       include_rts=False, exclude_replies=True).items(count)
tweets_lst = [[tweet.created_at, tweet.full_text.replace('\n', ' '), tweet.id] for tweet in tweets]

cols = "date text tweet_id".split()
tweets_df = pd.DataFrame(tweets_lst, columns=cols)

In [ ]:
tweets_df.head(4).append(tweets_df.tail(4))

,date,text,tweet_id
0,2021-07-25 11:05:37,Kenyataan Akhbar KPK 25 Julai 2021 – Situasi S...,1419252493377232899
1,2021-07-25 10:51:50,"COVID-19: 25 Julai 2021 J kes sembuh:9,683 J ...",1419249023509098497
0,2021-07-25 11:05:37,Kenyataan Akhbar KPK 25 Julai 2021 – Situasi S...,1419252493377232899
1,2021-07-25 10:51:50,"COVID-19: 25 Julai 2021 J kes sembuh:9,683 J ...",1419249023509098497


In [ ]:
bool_rel_idx = tweets_df.text.str.contains('terkini covid|^COVID-19[:;]', regex=True, case=False)
bool_nonrel_idx = tweets_df.text.str.contains('negeri|kluster|vaccin|vaksin', regex=True, case=False)
tweets_df = tweets_df[bool_rel_idx]
tweets_df = tweets_df.loc[~bool_nonrel_idx, :]    # without copying   # ~ invert boolean, subset exclude [non relevant rows idx] [false]

tweets_df = tweets_df[::-1].reset_index(drop=True)     # reorder rows, reset index
#tweets_df['date'] = pd.to_datetime(tweets_df['date']).dt.date
tweets_df['date'] = pd.to_datetime(tweets_df['date'].dt.date)
tweets_df[['text','link']] = tweets_df['text'].str.split('https://', expand=True)

In [ ]:
# tweets_df['text'].iloc[0]

In [ ]:
tweets_df

,date,text,tweet_id,link
0,2021-07-25,"COVID-19: 25 Julai 2021 J kes sembuh:9,683 J ...",1419249023509098497,t.co/t1ZtOUr683


In [ ]:
df_lst = []

for i in range(len(tweets_df)):
  text = tweets_df.loc[i,'text']
  
  tweet_lst = re.findall(r"[0-9]+[,.]?[0-9]*[,.]?[0-9]*", text)
  col0_8_lst = tweet_lst[3:12]        # elemnt 4 until 12  # total 9  
  col17_20_lst = tweet_lst[-4:]

  importcase = re.search(r"(import)(.*?)(J)", text, re.IGNORECASE).group(0)                     # we only know something is in it
  import_total = re.search(r"[0-9]+", importcase).group(0)                                      # get the first number  # total 
  if import_total != '0':                                                                       # if not 0 
    import_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", importcase, re.IGNORECASE)               # get WN value
    import_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", importcase, re.IGNORECASE)             # get BWN value
    if import_wn_re is not None:                                                                 
      import_wn = import_wn_re.group(1)
      if import_bwn_re is None:
        import_bwn = str(int(import_total) - int(import_wn))
    if import_bwn_re is not None:
      import_bwn = import_bwn_re.group(1)
      if import_wn_re is None:
        import_wn = str(int(import_total) - int(import_bwn)) 
    if import_wn_re == import_bwn_re == None:                                                   # if both dont exist, assign empty string
      import_wn = import_bwn = '' 
  else: import_total = import_wn = import_bwn = '0'


  maticase = re.search(r"(mati|fataliti)(.*?)(J)", text, re.IGNORECASE).group(0)                # we only know something is in it
  mati_total = re.search(r"[0-9]+", maticase).group(0)                                          # get the first number  # total 
  if import_total != '0':                                                                       # if not 0 
    mati_wn_re = re.search(r"([0-9]+)(?:\s?)(?:WN)", maticase, re.IGNORECASE)
    mati_bwn_re = re.search(r"([0-9]+)(?:\s?)(?:BWN)", maticase, re.IGNORECASE)
    if mati_wn_re is not None:
      mati_wn = mati_wn_re.group(1)
      if mati_bwn_re is None:
        mati_bwn = str(int(mati_total) - int(mati_wn))
    if mati_bwn_re is not None:
      mati_bwn = mati_bwn_re.group(1)
      if mati_wn_re is None:
        mati_wn = str(int(mati_total) - int(mati_bwn)) 
    if mati_wn_re == mati_bwn_re == None:
      mati_wn = mati_bwn = '' 
  else: mati_total = mati_wn = mati_bwn = '0'

  col9_15_lst = [import_total, import_wn, import_bwn, mati_total, mati_wn, mati_bwn]

  cols_lst = col0_8_lst + col9_15_lst + col17_20_lst
  df_lst.append(cols_lst)

In [ ]:
df = pd.DataFrame(df_lst)
df.insert(0, 'date', tweets_df['date']) 
df['tweet_id'] = tweets_df['tweet_id']
df['link'] = tweets_df['link']
df

,date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,tweet_id,link
0,2021-07-25,"9,683","844,541",83.3,"17,045","1,013,438","160,903","17,039","13,169","3,870",6,4,2,92,90,2,"7,994",0.79,970,501,1419249023509098497,t.co/t1ZtOUr683


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe

sh = gc.open('malaysia covid dataset -- link tableau')
worksheet = sh.worksheet('case data')

def get_next_lastrow(worksheet):
  str_lst = list(filter(None, worksheet.col_values(1)))
  return str(len(str_lst)+1)
  
next_lastrow = int(get_next_lastrow(worksheet))

set_with_dataframe(worksheet, df, row=next_lastrow, col=1, include_index=False, include_column_header=False)

# wget

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from PIL import Image
from datetime import date

today = str(date.today())    # '2020-06-31'
yyyymmdd = today.split('-')  # ['2021', '06', '31']

dd = yyyymmdd[2]
mm = yyyymmdd[1]
ddmm = '{}{}'.format(yyyymmdd[2],yyyymmdd[1])

# 'https://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/06.06/situasi-terkini-covid-19-di-malaysia-09062021/taburankes-all.jpg'
# 'https://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/06.06/situasi-terkini-covid-19-di-malaysia-08062021/taburankes-all.jpg'
# 'https://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/07.07/situasi-terkini-covid-19-di-malaysia-01072021/taburankes-all.jpg'


url = 'https://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/{}.{}/situasi-terkini-covid-19-di-malaysia-{}2021/taburankes-all.jpg'.format(mm, mm, ddmm)

!wget -P './' {url}    # -O name.jpg

--2021-07-25 11:51:22--  https://covid-19.moh.gov.my/user/pages/02.terkini/01.2021/07.07/situasi-terkini-covid-19-di-malaysia-25072021/taburankes-all.jpg
Resolving covid-19.moh.gov.my (covid-19.moh.gov.my)... 45.124.98.9
Connecting to covid-19.moh.gov.my (covid-19.moh.gov.my)|45.124.98.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 710579 (694K) [image/jpeg]
Saving to: ‘./taburankes-all.jpg’

taburankes-all.jpg  100%[===================>] 693.92K   374KB/s    in 1.9s    

2021-07-25 11:51:26 (374 KB/s) - ‘./taburankes-all.jpg’ saved [710579/710579]



In [ ]:
try: 
  img = Image.open('./taburankes-all.jpg')
  img.verify()
  img_name = '{}-{}-2021'.format(dd,mm)
  # img_name = text.partition('malaysia-')[2].partition("/taburankes-all")[0]
  !mv "taburankes-all.jpg" {img_name}.jpg
  !cp {img_name}.jpg "/content/drive/MyDrive/covid taburan kes"
  with open("/content/drive/MyDrive/covid taburan kes/taburan kes.txt", "a") as f:
    f.write("{},{}\n".format(today,url))
except (IOError, SyntaxError) as e:
  print('Bad file') 